In [25]:
import pandas as pd
%matplotlib inline
import cufflinks as cf
from statsmodels.tsa.stattools import acf, pacf, kpss
from statsmodels.tsa.arima.model import ARIMA

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

In [2]:
#1a
# Read entire data frame, convert to data frame where only location_name is US
entire_df = pd.read_csv('target-hospital-admissions.csv')
US_df = entire_df[entire_df["location_name"] == "US"]
US_df = US_df[['date','value']] # Since all the values are in the US, only need date and value columns
US_df['date'] = pd.to_datetime(US_df['date']) # Ensuring that the strings in date column are DateTime objects
US_df.set_index('date', inplace=True) # Eliminate the leading entry column by setting date as the index column
US_df

,value
date,
2024-04-27,2337
2024-04-20,2860
2024-04-13,3957
2024-04-06,4951
2024-03-30,5445
...,...
2022-03-12,2223
2022-03-05,1889
2022-02-26,1669


In [3]:
US_df["value"].iplot(xTitle = "Date", yTitle = "Count", title = "Count of Influenza Cases in US Over Time")

In [4]:
#1b
#Histogram of US_df data, showing the distribution in various buckets
US_df["value"].iplot(kind = "histogram", xTitle = "Range of Counts", yTitle = "Frequency", title = "Distribution of Influenza Counts in US")

In [5]:
# Calculations for mean, median, and standard deviation using pandas built in mean, median, and st_dev functions
mean = US_df.mean()
print("Mean is:", float(mean))
median = US_df.median()
print("Median is:", float(median))
st_dev = US_df.std()
print("Standard deviation is:", float(st_dev))

Mean is: 4526.206896551724
Median is: 1895.5
Standard deviation is: 5796.186409696639


In [6]:
#1c
#Same process as getting US_df, except Vermont is the location needed
VT_df = entire_df[entire_df["location_name"] == "Vermont"]
VT_df = VT_df[['date','value']]
VT_df['date'] = pd.to_datetime(VT_df['date'])
VT_df.set_index('date', inplace=True)
VT_df

,value
date,
2024-04-27,6
2024-04-20,10
2024-04-13,16
2024-04-06,9
2024-03-30,15
...,...
2022-03-12,0
2022-03-05,0
2022-02-26,1


In [7]:
#Creating new data frame to have both US and VT data
#This data frame will have exact same index as US_df. Has two columns: one with US data and one with VT data
US_VT_df = pd.DataFrame()
US_VT_df.index = US_df.index
US_VT_df["US"] = US_df["value"]
US_VT_df["Vermont"] = VT_df["value"]
US_VT_df

,US,Vermont
date,,
2024-04-27,2337,6
2024-04-20,2860,10
2024-04-13,3957,16
2024-04-06,4951,9
2024-03-30,5445,15
...,...,...
2022-03-12,2223,0
2022-03-05,1889,0
2022-02-26,1669,1


In [8]:
#Use of second y axis for Vermont data
US_VT_df.iplot(xTitle = "Date", yTitle = "Count for US", title = "Count of Influenza Cases in US and Vermont Over Time", secondary_y = "Vermont", secondary_y_title = "Count for Vermont")

In [9]:
#2a
#Various rolling average series of window sizes 3, 5, 7, 10
rolling_average_3_series = US_df["value"].rolling(window = 3).mean()
rolling_average_5_series = US_df["value"].rolling(window = 5).mean()
rolling_average_7_series = US_df["value"].rolling(window = 7).mean()
rolling_average_10_series = US_df["value"].rolling(window = 10).mean()

In [10]:
#Adding rolling average series to new data frame with US weekly counts
US_rollingav_df = pd.DataFrame()
US_rollingav_df.index = US_df.index
US_rollingav_df["US Weekly Count"] = US_df["value"]
US_rollingav_df["Rolling Average, Window of 3"] = rolling_average_3_series
US_rollingav_df

,US Weekly Count,"Rolling Average, Window of 3"
date,,
2024-04-27,2337,NaN
2024-04-20,2860,NaN
2024-04-13,3957,3051.333333
2024-04-06,4951,3922.666667
2024-03-30,5445,4784.333333
...,...,...
2022-03-12,2223,2681.333333
2022-03-05,1889,2290.666667
2022-02-26,1669,1927.000000


In [11]:
US_rollingav_df.iplot(xTitle = "Date", yTitle = "Count", title = "Weekly Count vs. Rolling Average of Influenza Cases in US")

In [12]:
#2b
#Various series of rolling standard deviations with different window sizes
rolling_stdev_3_series = US_df["value"].rolling(window = 3).std()
rolling_stdev_5_series = US_df["value"].rolling(window = 5).std()
rolling_stdev_7_series = US_df["value"].rolling(window = 7).std()
rolling_stdev_10_series = US_df["value"].rolling(window = 10).std()

In [13]:
#2c
#Equal probability binning with 10 bins
equal_prob_df = pd.DataFrame()
equal_prob_df.index = US_df.index
equal_prob_df["bin"] = pd.qcut(US_df["value"], q = 10, labels = False) # Use of pandas qcut function to create ten bins of equal size
equal_prob_df

,bin
date,
2024-04-27,5
2024-04-20,6
2024-04-13,7
2024-04-06,7
2024-03-30,7
...,...
2022-03-12,5
2022-03-05,4
2022-02-26,4


In [14]:
#Value counts and sort index functions to count each instance of bin 1-10 and sorting in ascending order
equal_prob_df['bin'].value_counts().sort_index().iplot(kind='bar', xTitle = "Bin Number", yTitle = "Frequency", title = "Equal Probability Binning of US Data")

In [15]:
#Equal width binning with 10 bins
equal_width_df = pd.DataFrame()
equal_width_df.index = US_df.index
equal_width_df['bin'] = pd.cut(US_df["value"], bins = 10, labels = False) # Cut function with specified 10 bins
equal_width_df

,bin
date,
2024-04-27,0
2024-04-20,0
2024-04-13,1
2024-04-06,1
2024-03-30,1
...,...
2022-03-12,0
2022-03-05,0
2022-02-26,0


In [16]:
#Value counts and sort index functions to count each instance of bin 1-10 and sorting in ascending order
equal_width_df['bin'].value_counts().sort_index().iplot(kind='bar', xTitle = "Bin Number", yTitle = "Frequency", title = "Equal Width Binning of US Data")

In [17]:
#3a
#Stationarity Test with Kwiatkowski-Phillips-Schmidt-Shin Approach

kpss_test = kpss(US_df['value'], regression = 'c', nlags = 20) # Test itself
kpss_output = pd.Series(kpss_test[0:3], index=['Test Statistic', 'p-value', 'Lags Used']) # Creating initial series with first outputs of test
for key, value in kpss_test[3].items(): # Can iterate to populate critical value indexes and values into series
    kpss_output[f'Critical Value ({key})'] = value

kpss_output
# With test statistic value below critical values at all percentage levels, data is likely stationary
# Since data is stationary no need for differencing, d value of 0 for ARIMA model

/var/folders/q2/pyrq5k_s7gz_4v527qy_7lcc0000gn/T/ipykernel_72702/3210603369.py:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




Test Statistic            0.123654
p-value                   0.100000
Lags Used                20.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64

In [18]:
lag_acf = acf(US_df['value'], nlags = 20) # Use of acf function and converting to series
lag_acf_series = pd.Series(lag_acf)
lag_acf_series.iplot(kind = "bar", yTitle = "ACF", xTitle = "Lag", title = "ACF Analysis of US Data")
# At lag 9 is when ACF is closest to 0, will use q value of 9 for ARIMA model

In [19]:
lag_pacf = pacf(US_df['value'], nlags = 20, method = "ywm") # Use of pacf function and converting to series
lag_pacf_series = pd.Series(lag_pacf)
lag_pacf_series.iplot(kind = "bar", xTitle = "Lag", yTitle = "PACF", title = 'PACF Analysis of US Data')
# Lag 2 is last lag spike before flattening out. Will use p value of 2 in ARIMA model

In [20]:
#3b

overall_model = ARIMA(US_df['value'], order = (2, 0, 9)) # Parameters based upon stationary, ACF, and PACF tests
overall_model_fit = overall_model.fit()

print(overall_model_fit.summary())

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency -1W-SAT will be used.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency -1W-SAT will be used.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred 

                               SARIMAX Results                                
Dep. Variable:                  value   No. Observations:                  116
Model:                 ARIMA(2, 0, 9)   Log Likelihood                -985.738
Date:                Tue, 21 May 2024   AIC                           1997.475
Time:                        20:06:08   BIC                           2033.272
Sample:                             0   HQIC                          2012.007
                                - 116                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4526.1541    952.305      4.753      0.000    2659.670    6392.638
ar.L1          1.9693      0.033     59.817      0.000       1.905       2.034
ar.L2         -0.9811      0.031    -31.265      0.0

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [21]:
#3c
#Creating truncated df by going from start til first week of December
US_truncated_df = US_df.loc["2022-02-12":"2023-12-2"]
US_truncated_df

,value
date,
2023-12-02,5752
2023-11-25,4240
2023-11-18,3422
2023-11-11,2695
2023-11-04,1974
...,...
2022-03-12,2223
2022-03-05,1889
2022-02-26,1669


In [22]:
# Creating 4 week forecast based on truncated df
truncated_model = ARIMA(US_truncated_df['value'], order = (2, 0, 9)) # Same p, d, q values as for the overall df
truncated_model_fit = truncated_model.fit()

forecast = truncated_model_fit.forecast(steps = 4) # Need the next 4 weeks of data


new_index = ["2023-12-09", "2023-12-16", "2023-12-23", "2023-12-30"]

forecast.index = new_index # Changing index and ensuring it is DateTime object
forecast.index = pd.to_datetime(forecast.index)
print(forecast)

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency -1W-SAT will be used.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency -1W-SAT will be used.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred 

2023-12-09    1232.386449
2023-12-16    1426.853187
2023-12-23    1648.304966
2023-12-30    1764.905296
Name: predicted_mean, dtype: float64


/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/Users/anshulchiranth/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [23]:
arima_df = pd.DataFrame()
arima_df = US_df.loc["2022-02-12": "2024-01-1"] # Copying entire US_df between specified dates
arima_df["Truncated Data"] = US_truncated_df['value'] # Add first series which is the truncated data
del arima_df['value'] # No need for this, as the real data will be split between two series
arima_df["ARIMA Prediction"] = forecast # Arima forecast added
arima_df["Actual Data"] = US_df['value'].loc["2023-12-09": "2024-01-1"] # Actual data added
arima_df

,Truncated Data,ARIMA Prediction,Actual Data
date,,,
2023-12-30,NaN,1764.905296,21030.0
2023-12-23,NaN,1648.304966,15134.0
2023-12-16,NaN,1426.853187,9886.0
2023-12-09,NaN,1232.386449,7178.0
2023-12-02,5752.0,NaN,NaN
...,...,...,...
2022-03-12,2223.0,NaN,NaN
2022-03-05,1889.0,NaN,NaN
2022-02-26,1669.0,NaN,NaN


In [24]:
arima_df.iplot(kind = "line", xTitle = "Date", yTitle = "Count", title = "ARIMA Prediction vs. Actual Data of December 2023 Influenza Cases")